
- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [ ]:
import pandas as pd
import numpy as np
import os
import ast
import json
import re
from pprint import pprint
from pandas import json_normalize
import quandl

In [ ]:
# load one file first to see what type of data you're dealing with and what attributes it has

In [ ]:


# Specify the directory you want to explore
directory = '/Users/jorgen/Documents/LHL/Supervised-Learning-main/data'

# Use os.listdir to get files
files = os.listdir(directory)

# Print all files
for file in files:
    print(file)

In [ ]:
import json
from pprint import pprint

# Path to the JSON file
file_path = '/Users/jorgen/Documents/LHL/Supervised-Learning-main/data/AZ_Phoenix_2.json'

# Load and inspect the content of the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)
    pprint(data)  # Pretty print the JSON data to inspect its structure

At this point, ensure that you have all sales in a dataframe.
- Is each cell one value, or do some cells have lists?
- Maybe the "tags" will help create some features.
- What are the data types of each column?
- Some sales may not actually include the sale price.  These rows should be dropped.
- Some sales don't include the property type.
- There are a lot of None values.  Should these be dropped or replaced with something?

In [ ]:
# Directory containing the JSON files
directory = '/Users/jorgen/Documents/LHL/Supervised-Learning-main/data'

# List to hold each DataFrame
dfs = []

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.json'):  # Ensure processing only JSON files
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            # Check if the 'sales' key exists in the JSON data
            if 'sales' in data:
                df = pd.json_normalize(data, record_path=['sales'])
            else:
                # If 'sales' is not a key, perhaps normalize the entire JSON or another part
                df = pd.json_normalize(data)  # Adjust this according to your JSON structure
            dfs.append(df)

# Concatenate all dataframes into one
all_sales_df = pd.concat(dfs, ignore_index=True)

In [ ]:
# Reading the CSV data from the specified path into pandas DataFrame `X_train`
X_train = pd.read_csv('/Users/jorgen/Documents/LHL/Supervised-Learning-main/dataframe.csv')

# Iterating over each column present in the DataFrame
for column in X_train.columns:
    # Using the `apply` function to go through each cell in the column.
    # Apply function takes a lambda function which checks if the cell content is a list.
    # `any()` method checks if there is any cell in the column that satisfies the condition.
    has_list = X_train[column].apply(lambda x: isinstance(x, list)).any()

    # If a list is found in the column (i.e., if `has_list` is `True`),
    # then print the column name.
    if has_list:
        print(f"Column '{column}' contains a list")

In [ ]:
# If a 'tags' column exists, try printing out unique set of tags
if 'tags' in X_train.columns:
    unique_tags = set(tag for tags in X_train['tags'].dropna() for tag in tags)
    print(f"Unique tags: {unique_tags}")

In [ ]:
# Print types of each column
print(X_train.dtypes)

In [ ]:
# Check if a 'sale_price' column exists and if it contains NaN values
if 'sale_price' in X_train.columns:
    print(f"Number of rows before drop: {len(X_train)}")
    X_train = X_train.dropna(subset=['sale_price'])
    print(f"Number of rows after drop: {len(X_train)}")

In [ ]:
# Check if a 'property_type' column exists and if it contains NaN values
if 'property_type' in X_train.columns:
    property_type_nan_count = X_train['property_type'].isna().sum()
    print(f"Number of rows without property_type: {property_type_nan_count}")

In [ ]:
# Count the number of None values per column
none_count = X_train.isnull().sum()
print(none_count)

In [ ]:
# load and concatenate data here
# drop or replace values as necessary

In [ ]:
data_folder = '/Users/jorgen/Documents/LHL/Supervised-Learning-main/data'
json_files = [pos_json for pos_json in os.listdir(data_folder) if pos_json.endswith('.json')]

data_frames = []  # Collate all the DataFrames here

# Load each JSON file and convert it to pandas DataFrame
for file in json_files:
    file_path = os.path.join(data_folder, file)
    with open(file_path, 'r') as json_file:
        json_data = json.load(json_file)
        if isinstance(json_data, dict):
            # Flatten nested dictionary structures if 'data' is a dictionary
            json_df = json_normalize(json_data)
        else:
            # 'data' is not a dict, so it should be a list or scalar
            if isinstance(json_data, list):
                # Flatten list of dictionaries if 'data' is a list
                json_df = json_normalize(json_data)
            else:
                # 'data' is a scalar, so convert it into a DataFrame
                json_df = pd.DataFrame(data=[json_data], columns=['data'])
        data_frames.append(json_df)

# Concatenate all DataFrames
df = pd.concat(data_frames, ignore_index=True)

# Show the first few rows of the DataFrame
print(df.head())

In [ ]:
# 'Explode' the column of lists into separate rows
exploded_df = df.explode('data.results')

# Now, 'data.results' is a single dictionary per row,
# so we can convert these dictionaries into separate columns
results_df = exploded_df['data.results'].apply(pd.Series)

# Concatenate the original DataFrame with the new 'results' DataFrame
flattened_df = pd.concat([exploded_df, results_df], axis=1)

# We don't need 'data.results' anymore as its content is now in separate columns
flattened_df = flattened_df.drop(columns=['data.results'])

# Show the first few rows of the new DataFrame
print(flattened_df.head())

In [ ]:
# Handle the 'location' column
location_df = flattened_df['location'].apply(pd.Series)

# If 'location' is a nested dictionary, we can flatten it further
address_df = location_df['address'].apply(pd.Series)

# Now, we concatenate the original DataFrame with the 'location' DataFrame and 'address' DataFrame
full_df = pd.concat([flattened_df, location_df, address_df], axis=1)

# Drop the original 'location' and 'address' columns
full_df = full_df.drop(columns=['location', 'address'])

# Show the first few rows
print(full_df.head())

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [ ]:
# OHE categorical variables here
# tags will have to be done manually

In [ ]:
num_city_cols = full_df['city'].nunique()
num_state_cols = full_df['state'].nunique()

In [ ]:
all_tags = set(tag for tags in full_df['tags'] if isinstance(tags, list) for tag in tags)
num_tag_cols = len(all_tags)

tag_freq = {}
for tags_list in full_df['tags']:
    if isinstance(tags_list, list):
        for tag in tags_list:
            if tag not in tag_freq:
                tag_freq[tag] = 0
            tag_freq[tag] += 1

high_freq_tags = {tag for tag, freq in tag_freq.items() if freq > 5}
num_high_freq_tags = len(high_freq_tags)

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE such as using central tendency?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- If you replace cities or states with numerical values, make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Drop columns that aren't needed.
- Don't keep the list price because it will be too close to the sale price.

# perform train test split here
# do something with state and city
# drop any other not needed columns

In [ ]:
#Splitting the data into Train and Test

from sklearn.model_selection import train_test_split

X = full_df.drop('list_price', axis=1)  # dropping the 'list_price' column as mentioned
y = full_df['list_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.columns)

In [ ]:
#X_train

I did not clean the data properly. For example, in the source, the office_name should be its own column with associated office data. The fips_code should be its own column, and the country name should be its own column. Currently, all that data is mashed into the same column, causing issues.

In [ ]:
#Adjusting Issues with coordinate column Containing format {'lon': -81.039568, 'lat': 33.983242}

# Direct access to 'lon' and 'lat' in dictionary
X_train['lon'] = X_train['coordinate'].apply(lambda x: x.get('lon', np.nan) if isinstance(x, dict) else np.nan)
X_train['lat'] = X_train['coordinate'].apply(lambda x: x.get('lat', np.nan) if isinstance(x, dict) else np.nan)

# Drop the original 'coordinate' column
X_train.drop('coordinate', axis=1, inplace=True)

In [ ]:
#Checking if it worked, Took a while but that worked

#X_train

In [ ]:
#Fixing County Column to Separate Relevant data into new columns

# Direct access to 'fips_code' and 'name' in the dictionary
X_train['fips_code'] = X_train['county'].apply(lambda x: x.get('fips_code', np.nan) if isinstance(x, dict) else np.nan)
X_train['county'] = X_train['county'].apply(lambda x: x.get('name', np.nan) if isinstance(x, dict) else np.nan)

In [ ]:
#Checking if it worked
#X_train

In [ ]:
#Cleaning the other_listing column to Separate data

# Define function to extract a key's value from a list of dictionaries
def extract_value(list_of_dicts, key):
    return [dic.get(key, np.nan) for dic in list_of_dicts if isinstance(dic, dict)]


# Start by cleaning 'other_listings' column
X_train['other_listings'] = X_train['other_listings'].apply(lambda x: x.get('rdc', []) if isinstance(x, dict) else [])

# Create new columns by extracting fields from each dictionary in the 'rdc' list
X_train['other_listing_key'] = X_train['other_listings'].apply(lambda x: extract_value(x, 'listing_key'))
X_train['other_listing_status'] = X_train['other_listings'].apply(lambda x: extract_value(x, 'status'))
X_train['other_listing_primary'] = X_train['other_listings'].apply(lambda x: extract_value(x, 'primary'))

# Check the lengths of dataframes
print(len(X_train))
print(len(X_train['other_listings']))
print(len(X_train['other_listing_key']))
print(len(X_train['other_listing_status']))
print(len(X_train['other_listing_primary']))

# Convert lists to string in new columns using apply
X_train['other_listing_key'] = X_train['other_listing_key'].apply(lambda x: str(x))
X_train['other_listing_status'] = X_train['other_listing_status'].apply(lambda x: ', '.join(str(i) for i in x))
X_train['other_listing_primary'] = X_train['other_listing_primary'].apply(lambda x: str(x))

In [ ]:
#X_train

In [ ]:
# Changing null values to 0 for price reduced amount. Not every house would have reduced their prices.

X_train['price_reduced_amount'] = X_train['price_reduced_amount'].fillna(0)

**STRETCH**

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here. Plan is to import interest rate data and build model around what housing factors lead to an increase in interest rates & what how interest rate data leads to an increase in housing prices 

In [ ]:
# Convert 'list_date' in X_train to datetime, ignoring errors and stripping time
X_train['list_date'] = pd.to_datetime(X_train['list_date'], errors='coerce').dt.date

# Now, if there are any None values, they will become NaT in the datetime column
# which you can handle according to your project needs (e.g., fill with a placeholder or leave as is)

In [ ]:
# Check how many null values are there in 'list_date'
null_dates_count = X_train['list_date'].isnull().sum()
print(f"Number of null dates in X_train: {null_dates_count}")

# Optionally, examine rows with null dates to decide further actions
null_dates_rows = X_train[X_train['list_date'].isnull()]
print(null_dates_rows)

In [ ]:
# Set your Quandl API key
quandl.ApiConfig.api_key = "evXR317_odmBCAMTi6ym"

# Fetch mortgage rate data from the Quandl API
mortgage_data = quandl.get("FMAC/30US", start_date="2001-01-01", end_date="2024-12-31")
mortgage_data.reset_index(inplace=True)
mortgage_data.rename(columns={'Date': 'Date_API', 'Value': 'Mortgage_Rate'}, inplace=True)

# Convert 'Date_API' to proper datetime format
mortgage_data['Date_API'] = pd.to_datetime(mortgage_data['Date_API'])

# Assuming 'list_date' in X_train is already loaded and needs to be in datetime format
X_train['list_date'] = pd.to_datetime(X_train['list_date'])

# Drop rows where 'list_date' is null
X_train.dropna(subset=['list_date'], inplace=True)

# Sort dataframes by the date columns to use merge_asof
X_train_sorted = X_train.sort_values('list_date')
mortgage_data_sorted = mortgage_data.sort_values('Date_API')

# Perform an as-of merge to align dates as closely as possible
X_train_merged = pd.merge_asof(X_train_sorted, mortgage_data_sorted, left_on='list_date', right_on='Date_API', direction='nearest')

# Drop the 'Date_API' column if it's no longer needed after the merge
X_train_merged.drop('Date_API', axis=1, inplace=True)

# Display the first few rows of the updated DataFrame to confirm changes
X_train_merged.head()

In [ ]:
print(X_train.columns)

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

In [ ]:
# Statistics summary
X_train.describe()

In [ ]:
# Histograms for each numerical variable
X_train.hist(bins=50, figsize=(20, 15))

In [ ]:
#identifying column names
print(X_train.dtypes)

In [ ]:
#first square bracket means I want to create condition, second square bracket indicates exact conditions I want ot see

X_train_merged[['Mortgage_Rate', 'price_reduced_amount']]

In [ ]:
print(type(X_train['city'].iloc[0]))
print(type(X_train['price_reduced_amount'].iloc[0]))

In [ ]:
print(X_train['county'].isnull().values.any())
print(X_train['price_reduced_amount'].isnull().values.any())

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized

In [ ]:
# Specify the path 
file_path = "/Users/jorgen/Documents/LHL/Supervised-Learning-main/processed/processed.csv"

# Check if the directory exists, and create it if it does not
directory = "/Users/jorgen/Documents/LHL/Supervised-Learning-main/processed"
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the merged DataFrame to a CSV file at the specified path
X_train_merged.to_csv(file_path, index=False)